In [14]:
#!/usr/bin/env python
# coding: utf-8

# ==================================================================== #
# author: Yung-Hsin Chen                                               #
# copyright: Copyright 2019, Thermal and Statistical Physics Project   #
# credit: Mei-Yan Sam, Sing-Hong Chen                                  #                                              
# ==================================================================== #

# import modules
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import math

## Hamiltonian

perform a Monte Carlo simulation of the 2D Ising model in zero magnetic field.
$$
  H = -J \sum_{\langle i,j\rangle} \sigma_i \sigma_j,
$$
where $\sigma=\pm 1$ and the sum is over nearest-neighbor pairs of sites on a square lattice of $N = L \times L$ sites.

In [19]:
# set t interval = 0.02
J = 1.0
K = 1.0

def lattice():
    La = np.zeros((L, L))
    for i in range(L):
        for j in range(L):
            La[i][j] = 1
    return La

# Calculating the Hamiltonian
def energy(LA, L):     
    E_total = []
    for i in range(0, L, 1):
        for j in range(0, L, 1):
            E = LA[i][j] * LA[(i+1)%L][j] + LA[i][j] * LA[i][(j+1)%L]
            E_total.append(E)
    Energy = - J * sum(E_total) 
    return Energy

def dE(s, L, i, j):
    NB = s[(i+1)%L,j]+s[i,(j+1)%L]+s[(i-1)%L,j]+s[i,(j-1)%L]
    dE = 2 * s[i,j] * NB
    return dE

def magnet(s, L):
    M = 0.0
    for i in range(L):
        for j in range(L):
            M += s[i,j]
    return M

## Sequential update

In [20]:
def Monte_Carlo_seq(T, L, N_MC, spins):
    # initialize measurements
    beta = 1.0/(K*T)
    E_sum = 0.0
    E_sqr_sum = 0.0
    M_sum = 0.0
    M_sqr_sum = 0.0
    accept = 0

    # main MC loop
    for n in range(N_MC):
        # sequential update
        for i in range(L):
            for j in range(L):
                if np.random.random() < np.exp(- beta * dE(spins, L, i, j)):
                    spins[i,j] = -spins[i,j]
                    accept += 1
        # measurements
        E = energy(spins, L)
        E_sum += E
        E_sqr_sum += E*E
        m = magnet(spins, L)
        M_sum += np.abs(m) # for <|m|>
        M_sqr_sum += m*m
        
    # average
    UPDATES = N_MC
    acceptRatio = accept/UPDATES/(L*L)
    E = E_sum/UPDATES
    E_sqr = E_sqr_sum/UPDATES
    C = (E_sqr-E*E)/(K*T*T)
    M = M_sum/UPDATES
    M_sqr = M_sqr_sum/UPDATES 
    chi = (M_sqr-M*M)/(K*T)
    return E, C, M, chi, acceptRatio

T_0 = 1
dT = 0.05
N_sim = 80
N_MC = 10000

# plot MC (sequential)                      
def data_seq(L):
    T_list_seq = []
    e_list_seq = []
    c_list_seq = []
    m_list_seq = []
    chi_list_seq = []
    spins = np.ones( [L, L], int)
    for n in range(N_sim):
        T = T_0 + n * dT
        E, C, M, chi, acceptRatio = Monte_Carlo_seq(T, L, N_MC, spins)
        T_list_seq += [T]
        e_list_seq += [E/L/L]
        c_list_seq += [C/L/L]
        m_list_seq += [M/L/L]
        chi_list_seq += [chi/L/L]
    return T_list_seq, e_list_seq, c_list_seq, m_list_seq, chi_list_seq

## Random update

In [29]:
def Monte_Carlo_rd(T, L, N_MC, spins):
    # initialize measurements
    K = 1
    beta = 1.0/(K*T)
    E_sum = 0.0
    E_sqr_sum = 0.0
    M_sum = 0.0
    M_sqr_sum = 0.0
    N_MC = 10000
    accept = 0.0

    # main MC loop
    for n in range(N_MC):
        # random update
        for update in range(L*L):
            i = np.random.randint(L)
            j = np.random.randint(L)
            if np.random.random() < np.exp(- beta * dE(spins, L, i, j)):
                spins[i,j] = -spins[i,j]
                accept += 1
        # measurements
        E = energy(spins, L)
        E_sum += E
        E_sqr_sum += E*E
        m = magnet(spins, L)
        M_sum += np.abs(m) # for <|m|>
        M_sqr_sum += m*m
        
    # average
    UPDATES = N_MC
    acceptRatio = accept/UPDATES/(L*L)
    E = E_sum/UPDATES
    E_sqr = E_sqr_sum/UPDATES
    C = (E_sqr-E*E)/(K*T*T)
    M = M_sum/UPDATES
    M_sqr = M_sqr_sum/UPDATES 
    chi = (M_sqr-M*M)/(K*T)
    return E, C, M, chi, acceptRatio

T_0 = 1
dT = 0.05
N_sim = 80
N_MC = 10000

# plot MC (random)                      
def data_rd(L):
    T_list_rd = []
    e_list_rd = []
    c_list_rd = []
    m_list_rd = []
    chi_list_rd = []
    spins = np.ones( [L, L], int)
    for n in range(N_sim):
        T = T_0 + n * dT
        E, C, M, chi, acceptRatio = Monte_Carlo_rd(T, L, N_MC, spins)
        T_list_rd += [T]
        e_list_rd += [E/L/L]
        c_list_rd += [C/L/L]
        m_list_rd += [M/L/L]
        chi_list_rd += [chi/L/L]
    return T_list_rd, e_list_rd, c_list_rd, m_list_rd, chi_list_rd

## Save sequantial update data

In [22]:
np.save('MC_seq_2.npy', data_seq(2))
np.save('MC_seq_4.npy', data_seq(4))
np.save('MC_seq_8.npy', data_seq(8))
np.save('MC_seq_16.npy', data_seq(16))
np.save('MC_seq_32.npy', data_seq(32))

## Upload sequential update data

In [30]:
np.save('MC_rd_2.npy', data_rd(2))
np.save('MC_rd_4.npy', data_rd(4))
np.save('MC_rd_8.npy', data_rd(8))
np.save('MC_rd_16.npy', data_rd(16))
np.save('MC_rd_32.npy', data_rd(32))